Primero armo la lista de municipios

In [3]:
ms = '''<select _ngcontent-c1="" class="form-control ng-pristine ng-valid ng-touched">
                      <option _ngcontent-c1=""></option>
                      <!----><option _ngcontent-c1="" value="47">AVIA TERAI</option><option _ngcontent-c1="" value="2">BARRANQUERAS</option><option _ngcontent-c1="" value="5">BASAIL</option><option _ngcontent-c1="" value="48">CAMPO LARGO</option><option _ngcontent-c1="" value="16">CAPITAN SOLARI</option><option _ngcontent-c1="" value="33">CHARADAI</option><option _ngcontent-c1="" value="51">CHARATA</option><option _ngcontent-c1="" value="65">CHOROTIS</option><option _ngcontent-c1="" value="31">CIERVO PETISO</option><option _ngcontent-c1="" value="6">COLONIA BENITEZ</option><option _ngcontent-c1="" value="15">COLONIA ELISA</option><option _ngcontent-c1="" value="9">COLONIA POPULAR</option><option _ngcontent-c1="" value="17">COLONIAS UNIDAS</option><option _ngcontent-c1="" value="57">CONCEP.DEL BERMEJO</option><option _ngcontent-c1="" value="38">CORONEL DU GRATY</option><option _ngcontent-c1="" value="49">CORZUELA</option><option _ngcontent-c1="" value="32">COTE LAI</option><option _ngcontent-c1="" value="36">ENRIQUE URIEN</option><option _ngcontent-c1="" value="69">EL ESPINILLO</option><option _ngcontent-c1="" value="61">EL SAUZALITO</option><option _ngcontent-c1="" value="4">FONTANA</option><option _ngcontent-c1="" value="63">FUERTE ESPERANZA</option><option _ngcontent-c1="" value="27">G. J. DE SAN MARTIN</option><option _ngcontent-c1="" value="54">GANCEDO</option><option _ngcontent-c1="" value="53">GENERAL CAPDEVILA</option><option _ngcontent-c1="" value="52">GENERAL PINEDO</option><option _ngcontent-c1="" value="23">GENERAL VEDIA</option><option _ngcontent-c1="" value="55">HERMOSO CAMPO</option><option _ngcontent-c1="" value="20">ISLA DEL CERRITO</option><option _ngcontent-c1="" value="64">JUAN JOSE CASTELLI</option><option _ngcontent-c1="" value="10">LAGUNA BLANCA</option><option _ngcontent-c1="" value="30">LAGUNA LIMPIA</option><option _ngcontent-c1="" value="11">LAPACHITO</option><option _ngcontent-c1="" value="40">LA CLOTILDE</option><option _ngcontent-c1="" value="25">LA EDUVIGIS</option><option _ngcontent-c1="" value="14">LA ESCONDIDA</option><option _ngcontent-c1="" value="22">LA LEONESA</option><option _ngcontent-c1="" value="41">LA TIGRA</option><option _ngcontent-c1="" value="13">LA VERDE</option><option _ngcontent-c1="" value="50">LAS BREÑAS</option><option _ngcontent-c1="" value="18">LAS GARCITAS</option><option _ngcontent-c1="" value="21">LAS PALMAS</option><option _ngcontent-c1="" value="59">LOS FRENTONES</option><option _ngcontent-c1="" value="44">MACHAGAI</option><option _ngcontent-c1="" value="7">MARGARITA BELEN</option><option _ngcontent-c1="" value="12">MAKALLE</option><option _ngcontent-c1="" value="67">MIRAFLORES</option><option _ngcontent-c1="" value="62">MISION NUEVA POMPEYA</option><option _ngcontent-c1="" value="46">NAPENAY</option><option _ngcontent-c1="" value="26">PAMPA ALMIRON</option><option _ngcontent-c1="" value="29">PAMPA DEL INDIO</option><option _ngcontent-c1="" value="58">PAMPA DEL INFIERNO</option><option _ngcontent-c1="" value="19">PCIA. DE LA PLAZA</option><option _ngcontent-c1="" value="42">PCIA. ROQUE S.PEÑA</option><option _ngcontent-c1="" value="28">PRESIDENCIA ROCA</option><option _ngcontent-c1="" value="24">PUERTO BERMEJO</option><option _ngcontent-c1="" value="66">PUERTO EVA PERON</option><option _ngcontent-c1="" value="8">PUERTO TIROL</option><option _ngcontent-c1="" value="3">PUERTO VILELAS</option><option _ngcontent-c1="" value="43">QUITILIPI</option><option _ngcontent-c1="" value="1">RESISTENCIA</option><option _ngcontent-c1="" value="56">SANTA SYLVINA</option><option _ngcontent-c1="" value="60">TACO POZO</option><option _ngcontent-c1="" value="45">TRES ISLETAS</option><option _ngcontent-c1="" value="37">VILLA ANGELA</option><option _ngcontent-c1="" value="34">VILLA BERTHET</option><option _ngcontent-c1="" value="68">VILLA RIO BERMEJITO</option><option _ngcontent-c1="" value="35">SAMUHU</option><option _ngcontent-c1="" value="39">SAN BERNARDO</option>
                    </select>'''
import re
ids = [int(id) for id in re.findall(r'\"(\d+)\"', ms)]
nombres = re.findall(r'>([A-ZÑ\. ]+)<', ms)
muni_nombre = {ids[i]: nombres[i] for i in range(len(ids))}
nombre_muni = {nombres[i]: ids[i]  for i in range(len(ids))}

In [ ]:
import pandas as pd
df = pd.DataFrame({'id':ids, 'nombre':nombres})
df = df.sort_values('id')
df.to_csv('municipios.csv', index=False)

Corro el scrapper (`scrapper.php`) y obtengo `resultados.json` con los resultados por mesa. Los cargo y los proceso.

In [1]:
import json
with open('resultados.json') as data_file:    
    resultados = json.load(data_file)

In [2]:
import pandas as pd
columnas = ['mesa', 'circuito', 'municipio', 'municipio_nombre', 'electronica', 'cargada']
cols = {col: [] for col in columnas}
res = {}

for muni, mesas in resultados.items():
    for mesa in mesas:
        mesa_id = int(mesa['m'].split('-')[0])
        cols['mesa'].append(mesa_id)
        cols['municipio'].append(int(muni))
        cols['municipio_nombre'].append(muni_nombre[int(muni)])
        cols['electronica'].append(mesa['e'])
        res = mesa['resultado']
        cols['circuito'].append(res['circ'])
        if not res['votosvalidos'] and not res['votosnovalidos']:
            cols['cargada'].append(0)
            n = len(cols['cargada'])
            for l in cols.values():
                if len(l) < n: l.append(0)
            continue
        cols['cargada'].append(1)
        if not res['votosvalidos'] or not res['votosnovalidos']:
            print(res)
        for votos in res['votosvalidos']:
            colname = 'G'+str(votos['p'])
            if colname not in cols:
                cols[colname] = []
                columnas.append(colname)
            cols[colname].append(votos['vd'])
        for colname, votos in res['votosnovalidos'][0].items():
            colname = colname[:-3]
            if colname not in cols:
                cols[colname] = []
                columnas.append(colname)
            cols[colname].append(votos)
df = pd.DataFrame(cols, columns=columnas)
df = df.sort_values(['municipio', 'circuito', 'mesa'])
df['munimesa'] = [str(muni)+'.'+str(mesa) for muni, mesa in zip(df['municipio'], df['mesa'])]
df = df.set_index('munimesa')
res_df = df.copy()
res_df['femenina'] = [int(x>=1000) for x in res_df['mesa']]
columnas.insert(4, 'femenina')
res_df = res_df[columnas]

NameError: name 'muni_nombre' is not defined

Le agrego los totales de votos al final.

In [10]:
res_df['validos'] = [sum(row[7:17]) for _, row in res_df.iterrows()]
res_df['totales'] = [sum(row[7:21]) for _, row in res_df.iterrows()]
res_df.to_csv('resultados.csv', index=False)

Buscamos circuitos en los que la proporción de mesas cargadas con BUE esté entre 0 y 1 (no inclusive). Es decir, circuitos que son experimentos naturales de efecto del voto electrónico.

In [11]:
df = res_df
df = df[df['cargada'] == 1]
bue = [(round(100*g['electronica'].mean()), circ, len(g)) for circ, g in df.groupby('circuito')]
list(filter(lambda x: 0<x[0]<100, sorted(bue)))

[(16.0, '16A', 90),
 (27.0, '62', 84),
 (34.0, '119', 85),
 (35.0, '104', 63),
 (38.0, '13A', 42),
 (38.0, '88', 79),
 (39.0, '136', 90),
 (43.0, '98', 75),
 (44.0, '117', 72),
 (60.0, '78', 57),
 (73.0, '8A', 75),
 (82.0, '22A', 28)]